# This notebook runs the models for Table 2 (lag 0) and Table 4 (lag 10+) with and without using APOE as a covariate

In [ ]:
# Install models
!pip install pandas==1.5.3
!pip install statsmodels
!pip install lifelines==0.26.4

In [ ]:
# Imports here.
import numpy as np
import pandas as pd
import os
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.multitest import fdrcorrection
from lifelines import CoxPHFitter

import warnings
warnings.filterwarnings("ignore")

In [ ]:
#Select NDDs
ndd_list = ['AD', 'DEM', 'PD']

In [ ]:
#Load list of medication codes
meds2 = pd.read_csv('meds_with_cleaned_name_MAY_05_2025.csv')
codes = list(set(list(meds2['cleaned_med'])))
print(len(codes))

In [ ]:
print(ndd_list)

In [ ]:
print(len(codes))

In [ ]:
timeline = 'ever taken'
drug = 'all'
model = f'icd10_{drug}'

# Change which lags you'd like to use here -- options are '0' (Table 2) or '10+' (Table 4)
lag_list = ['10+']

results = []
cov_list = []

for lag in lag_list:

    for ndd in ndd_list:

        #Load df
        df = pd.read_csv(f'AoU_{ndd}_with_icd10_with_APOE_MAY_12_2025.csv', parse_dates = True)

        # Find codes to use so we don't have to use EVERYTHING
        codes_with_data = []

        for code in codes:

            # Change if you'd like to run the code with or without APOE (second row includes APOE)
            m = df[['age_at_tenure', 'SEX', 'tenure', ndd, f'QC{lag}_' + code, 'QC0_A04', 'QC0_B02', 'QC0_B37', 'QC0_E10', 'QC0_E11', 'QC0_E27', 'QC0_E78', 'QC0_E87', 'QC0_F20', 'QC0_F31', 'QC0_F32', 'QC0_F33', 'QC0_F40', 'QC0_F41', 'QC0_F42', 'QC0_F43', 'QC0_F44', 'QC0_F45', 'QC0_F48', 'QC0_F50', 'QC0_F51', 'QC0_G40', 'QC0_G43', 'QC0_G47', 'QC0_H66', 'QC0_I10', 'QC0_I11', 'QC0_I12', 'QC0_I15', 'QC0_I20', 'QC0_I21', 'QC0_I25', 'QC0_I47', 'QC0_I48', 'QC0_I49', 'QC0_I50', 'QC0_I60', 'QC0_I61', 'QC0_I62', 'QC0_I63', 'QC0_I65', 'QC0_I66', 'QC0_I67', 'QC0_I69', 'QC0_I82', 'QC0_K04', 'QC0_K05', 'QC0_K20', 'QC0_K21', 'QC0_K22', 'QC0_K25', 'QC0_K31', 'QC0_K51', 'QC0_K59', 'QC0_K70', 'QC0_K71', 'QC0_K72', 'QC0_K73', 'QC0_K74', 'QC0_K75', 'QC0_K76', 'QC0_L25', 'QC0_L40', 'QC0_L50', 'QC0_M06', 'QC0_M13', 'QC0_M15', 'QC0_M16', 'QC0_M17', 'QC0_M18', 'QC0_M19', 'QC0_M32', 'QC0_M45', 'QC0_M79', 'QC0_M80', 'QC0_M81', 'QC0_M88', 'QC0_N04', 'QC0_N10', 'QC0_N18', 'QC0_N19', 'QC0_N30', 'QC0_N31', 'QC0_N32', 'QC0_N39', 'QC0_N40', 'QC0_N94']]
            #m = df[['age_at_tenure', 'SEX', 'tenure', ndd, f'QC{lag}_' + code, 'QC0_A04', 'QC0_B02', 'QC0_B37', 'QC0_E10', 'QC0_E11', 'QC0_E27', 'QC0_E78', 'QC0_E87', 'QC0_F20', 'QC0_F31', 'QC0_F32', 'QC0_F33', 'QC0_F40', 'QC0_F41', 'QC0_F42', 'QC0_F43', 'QC0_F44', 'QC0_F45', 'QC0_F48', 'QC0_F50', 'QC0_F51', 'QC0_G40', 'QC0_G43', 'QC0_G47', 'QC0_H66', 'QC0_I10', 'QC0_I11', 'QC0_I12', 'QC0_I15', 'QC0_I20', 'QC0_I21', 'QC0_I25', 'QC0_I47', 'QC0_I48', 'QC0_I49', 'QC0_I50', 'QC0_I60', 'QC0_I61', 'QC0_I62', 'QC0_I63', 'QC0_I65', 'QC0_I66', 'QC0_I67', 'QC0_I69', 'QC0_I82', 'QC0_K04', 'QC0_K05', 'QC0_K20', 'QC0_K21', 'QC0_K22', 'QC0_K25', 'QC0_K31', 'QC0_K51', 'QC0_K59', 'QC0_K70', 'QC0_K71', 'QC0_K72', 'QC0_K73', 'QC0_K74', 'QC0_K75', 'QC0_K76', 'QC0_L25', 'QC0_L40', 'QC0_L50', 'QC0_M06', 'QC0_M13', 'QC0_M15', 'QC0_M16', 'QC0_M17', 'QC0_M18', 'QC0_M19', 'QC0_M32', 'QC0_M45', 'QC0_M79', 'QC0_M80', 'QC0_M81', 'QC0_M88', 'QC0_N04', 'QC0_N10', 'QC0_N18', 'QC0_N19', 'QC0_N30', 'QC0_N31', 'QC0_N32', 'QC0_N39', 'QC0_N40', 'QC0_N94', 'e3/e4', 'e4/e4']]
            
            
            n=sum(m[f'QC{lag}_'+ code])
            df_pair = m[m[f'QC{lag}_'+ code]==1]
            n_pairs = sum(df_pair[ndd])
            if n == 0:
                pass
            elif n_pairs < 10:
                pass
            elif n == n_pairs:
                pass
            else:
                print(code)
                codes_with_data.append(code)

        print(ndd)
        print(len(codes_with_data))

        for code in codes_with_data:

            # Change if you'd like to run the code with or without APOE (second row includes APOE)
            m = df[['age_at_tenure', 'SEX', 'tenure', ndd, f'QC{lag}_' + code, 'QC0_A04', 'QC0_B02', 'QC0_B37', 'QC0_E10', 'QC0_E11', 'QC0_E27', 'QC0_E78', 'QC0_E87', 'QC0_F20', 'QC0_F31', 'QC0_F32', 'QC0_F33', 'QC0_F40', 'QC0_F41', 'QC0_F42', 'QC0_F43', 'QC0_F44', 'QC0_F45', 'QC0_F48', 'QC0_F50', 'QC0_F51', 'QC0_G40', 'QC0_G43', 'QC0_G47', 'QC0_H66', 'QC0_I10', 'QC0_I11', 'QC0_I12', 'QC0_I15', 'QC0_I20', 'QC0_I21', 'QC0_I25', 'QC0_I47', 'QC0_I48', 'QC0_I49', 'QC0_I50', 'QC0_I60', 'QC0_I61', 'QC0_I62', 'QC0_I63', 'QC0_I65', 'QC0_I66', 'QC0_I67', 'QC0_I69', 'QC0_I82', 'QC0_K04', 'QC0_K05', 'QC0_K20', 'QC0_K21', 'QC0_K22', 'QC0_K25', 'QC0_K31', 'QC0_K51', 'QC0_K59', 'QC0_K70', 'QC0_K71', 'QC0_K72', 'QC0_K73', 'QC0_K74', 'QC0_K75', 'QC0_K76', 'QC0_L25', 'QC0_L40', 'QC0_L50', 'QC0_M06', 'QC0_M13', 'QC0_M15', 'QC0_M16', 'QC0_M17', 'QC0_M18', 'QC0_M19', 'QC0_M32', 'QC0_M45', 'QC0_M79', 'QC0_M80', 'QC0_M81', 'QC0_M88', 'QC0_N04', 'QC0_N10', 'QC0_N18', 'QC0_N19', 'QC0_N30', 'QC0_N31', 'QC0_N32', 'QC0_N39', 'QC0_N40', 'QC0_N94']]
            #m = df[['age_at_tenure', 'SEX', 'tenure', ndd, f'QC{lag}_' + code, 'QC0_A04', 'QC0_B02', 'QC0_B37', 'QC0_E10', 'QC0_E11', 'QC0_E27', 'QC0_E78', 'QC0_E87', 'QC0_F20', 'QC0_F31', 'QC0_F32', 'QC0_F33', 'QC0_F40', 'QC0_F41', 'QC0_F42', 'QC0_F43', 'QC0_F44', 'QC0_F45', 'QC0_F48', 'QC0_F50', 'QC0_F51', 'QC0_G40', 'QC0_G43', 'QC0_G47', 'QC0_H66', 'QC0_I10', 'QC0_I11', 'QC0_I12', 'QC0_I15', 'QC0_I20', 'QC0_I21', 'QC0_I25', 'QC0_I47', 'QC0_I48', 'QC0_I49', 'QC0_I50', 'QC0_I60', 'QC0_I61', 'QC0_I62', 'QC0_I63', 'QC0_I65', 'QC0_I66', 'QC0_I67', 'QC0_I69', 'QC0_I82', 'QC0_K04', 'QC0_K05', 'QC0_K20', 'QC0_K21', 'QC0_K22', 'QC0_K25', 'QC0_K31', 'QC0_K51', 'QC0_K59', 'QC0_K70', 'QC0_K71', 'QC0_K72', 'QC0_K73', 'QC0_K74', 'QC0_K75', 'QC0_K76', 'QC0_L25', 'QC0_L40', 'QC0_L50', 'QC0_M06', 'QC0_M13', 'QC0_M15', 'QC0_M16', 'QC0_M17', 'QC0_M18', 'QC0_M19', 'QC0_M32', 'QC0_M45', 'QC0_M79', 'QC0_M80', 'QC0_M81', 'QC0_M88', 'QC0_N04', 'QC0_N10', 'QC0_N18', 'QC0_N19', 'QC0_N30', 'QC0_N31', 'QC0_N32', 'QC0_N39', 'QC0_N40', 'QC0_N94', 'e3/e4', 'e4/e4']]
            
            n=sum(m[f'QC{lag}_'+ code])
            df_pair = m[m[f'QC{lag}_'+ code]==1]
            n_pairs = sum(df_pair[ndd])

            cph = CoxPHFitter()
            cph.fit(m, duration_col = 'tenure', event_col = ndd, show_progress=False, step_size = 0.01) #baseline 0.01
            #cph.print_summary()
            #cph.plot()

            actual_p = cph._compute_p_values()
            results_df = cph.summary
            
            # Filter for significant covariates, e.g., p-value < 0.05
            significant_covariates = results_df[results_df['p'] < 0.05].index.tolist()
            #cov_list.append(code)
            cov_list.append(significant_covariates)
            
            results_df = results_df.reset_index()
            test = results_df.iloc[2]

            covariate = code
            HR = test['exp(coef)']
            ci_min = test['exp(coef) lower 95%']
            ci_max = test['exp(coef) upper 95%']
            p = actual_p[2]

            print(covariate, ndd, HR, ci_min, ci_max, p, n_pairs, n)
            results.append((covariate, ndd, model, timeline, lag, HR, ci_min, ci_max, p, n_pairs, n))
            
cox1 = pd.DataFrame(results, columns=('PRIOR','OUTCOME', 'MODEL','TIMELINE', 'LAG', 'HR', 'ci_min', "ci_max", 'P_VAL', "N_pairs", "N"))

In [ ]:
from collections import Counter

# Example nested list
nested_list = cov_list

# Flatten the nested list
flat_list = [item for sublist in nested_list for item in sublist]

# Count the items
counts = Counter(flat_list)

# Convert to a DataFrame
counts_df = pd.DataFrame(counts.items(), columns=['item', 'count'])

counts_df


In [ ]:
output = pd.concat([cox1])

#Adding FDR Correction

#Sort P-values
output = output.sort_values(by = "P_VAL")

#Drop Nan-values
output = output.dropna()

#FDR Correction
rejected, p_corr = fdrcorrection(output['P_VAL'], is_sorted=True)
output['P_CORR'] = p_corr
output['SIGNIFICANT'] = rejected

output

# Save output files

In [ ]:
import os
import subprocess
import numpy as np
import pandas as pd

In [ ]:
date = 'MAY_12_2025'
counts_df.to_csv(f'AoU_{date}_cox_covariates_lag10.csv', header = True, index = False)

In [ ]:
# This snippet assumes you run setup first

# This code saves your dataframe into a csv file in a "data" folder in Google Bucket

# Replace df with THE NAME OF YOUR DATAFRAME
my_dataframe = counts_df   

# Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
destination_filename = f'AoU_{date}_cox_covariates_lag10.csv'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# save dataframe in a csv file in the same workspace as the notebook
my_dataframe.to_csv(destination_filename, index=False)

# get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# copy csv file to the bucket
args = ["gsutil", "cp", f"./{destination_filename}", f"{my_bucket}/data/results/"]
output = subprocess.run(args, capture_output=True)

# print output from gsutil
output.stderr

In [ ]:
date = 'MAY_12_2025'
output.to_csv(f'AoU_{date}_results_UPDATED_with_ICD10_lag10.csv', header = True, index = False)

In [ ]:
# This snippet assumes you run setup first

# This code saves your dataframe into a csv file in a "data" folder in Google Bucket

# Replace df with THE NAME OF YOUR DATAFRAME
my_dataframe = output   

# Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
destination_filename = f'AoU_{date}_results_UPDATED_with_ICD10_lag10.csv'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# save dataframe in a csv file in the same workspace as the notebook
my_dataframe.to_csv(destination_filename, index=False)

# get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# copy csv file to the bucket
args = ["gsutil", "cp", f"./{destination_filename}", f"{my_bucket}/data/results/"]
output = subprocess.run(args, capture_output=True)

# print output from gsutil
output.stderr
